# Polars and SQL Example Notebook
Created 10/10/2022 by Timothy Green (tdgreen@outlook.com)

In [ ]:
import pyarrow
import numpy as np
import pandas as pd
import polars as pl

In [ ]:
%%sql sql_query <<

SELECT
    CASE
        WHEN A."SBCNT#" LIKE '%PLASMAN%'
            /* 
                Get everything between brackets in contract description
                and use as contract number
            */
            THEN REGEXP_SUBSTR(B.ODDESC, '(?<=\().*?(?=\))')
            /*
                Get all numberic values from contract number
                and use as contract number
            */
        ELSE REGEXP_SUBSTR(A."SBCNT#", '\d+')
    END                     AS CONTRACT_NUMBER
    ,A.SBINV                AS INVOICE_NO
    ,A.SBINDT               AS INVOICE_DATE
    ,CASE
        WHEN A.SBITEM LIKE 'CRC%'
            /*
                When the item number contains 'CRC'
                get all charactes after CRC
                and use as item number
            */
            THEN REGEXP_SUBSTR(A.SBITEM, '(?<=CRC)(.+)')
        ELSE A.SBITEM
    END                     AS ITEM_NO
    ,A.SBQSHP               AS QTY_SHIPPED
    ,E.IGCOST               AS ITEM_PURCHASE_COST
    ,E.IGCOST - F.OIRAMT    AS DEVIATED_PRICE
    ,(E.IGCOST - F.OIRAMT) 
        * A.SBQSHP          AS REBATE_AMOUNT
    ,C.RMNAME               AS CUSTOMER_NAME

FROM
    R50FILES.VSADETL AS A
        LEFT JOIN R50FILES.VCORBTH AS B
            ON  B.ODCMP = 1
            AND B."ODCNT#" = A."SBCNT#"
            AND B.ODVEND = A.SBVEND
        LEFT JOIN R50FILES.VARCUST AS C
            ON  C.RMCMP = 1
            AND C.RMCUST = A.SBCUST
        LEFT JOIN R50FILES.VSAHEAD AS D
            ON D.SACMP = 1
            AND D.SAORD = A.SBORD
            AND D.SABOCD = A.SBBOCD
        LEFT JOIN R50FILES.VINITVN AS E
            ON E.IGCOMP = 1
            AND E.IGLOC = '107'
            AND E.IGITEM = A.SBITEM
            AND E.IGVND = A.SBVEND
        LEFT JOIN R50FILES.VCORBTD AS F
            ON F.OICMP = 1
            AND F."OICNT#" = A."SBCNT#"
            AND F.OIVEND = A.SBVEND
            AND F.OIITEM = A.SBITEM
WHERE
        A.SBVEND = '22401'
    AND A.SBTYPE = 'O'
    AND A.SBRAMT > 0
    AND A.SBINDT BETWEEN
        DECIMAL(LAST_DAY(CURRENT_DATE) - 2 MONTH + 1 DAY) 
            AND DECIMAL(LAST_DAY(CURRENT_DATE) - 1 MONTH)
FOR READ ONLY


In [ ]:
def rebate_report(df : pl.DataFrame) -> pd.DataFrame:
    return (
        df
            .lazy()
            .with_columns(
                invoice_date = pl.col('invoice_date')
                    .cast(pl.Int64)
                    .cast(pl.Utf8)
                    .str.strptime(pl.Date, fmt="%Y%m%d", strict=False),
                po_no_to_crc = pl.lit(None),
            )
            .with_columns(
                [pl.col(column).fill_null(0).cast(pl.Int64) for column in [
                    'qty_shipped',
                ]]
            )
            .with_columns(
                [pl.col(column).fill_null(0).cast(pl.Float64).round(2) for column in [
                    'item_purchase_cost',
                    'deviated_price',
                    'rebate_amount',
                ]]
            )
            .rename({
                'contract_number'       : 'Deviation#',
                'customer_name'         : 'End User',
                'invoice_no'            : 'Invoice# to User',
                'invoice_date'          : 'Date Sold',
                'item_no'               : 'Item#',
                'qty_shipped'           : 'Qty. Sold (ea.)',
                'item_purchase_cost'    : 'PO Price',
                'deviated_price'        : 'Deviated Price',
                'rebate_amount'         : 'Rebate due',
                'po_no_to_crc'          : 'PO# to CRC',
            })
            .select(
                [pl.col(column) for column in [
                    'Deviation#',
                    'PO# to CRC',
                    'End User',
                    'Invoice# to User',
                    'Date Sold',
                    'Item#',
                    'Qty. Sold (ea.)',
                    'PO Price',
                    'Deviated Price',
                    'Rebate due',
                ]]
            )
            .collect()
            .to_pandas()
    )